<h1><center>The Battle of Neighborhoods</center></h1>

This **capstone project** has been submitted as part of the requirements for completion of the **IBM Data Science Professional Certificate** on Coursera. In general, this project would be encompassing a series of Data Science techniques, including, but not limited to, Web Scraping (using BeautifulSoup and Requests), Data Cleaning, Data Wrangling and Machine Learning (K-Means clustering algorithm).

### Importing required libraries and packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


### Scraping web for neighborhood data of Berlin (WikiPedia)

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []

In [3]:
# append the data into the list
neighborhoodList.clear()

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
berlin_df = df1.reset_index(drop=True)
berlin_df.head()

,Neighborhood
0,Adlershof
1,Alt-Hohenschönhausen
2,Alt-Treptow
3,Altglienicke
4,Baumschulenweg


In [37]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head()

(96, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.43779,13.54778
1,Alt-Hohenschönhausen,52.54706,13.50055
2,Alt-Treptow,52.49350,13.45711
3,Altglienicke,52.42006,13.53969
4,Baumschulenweg,52.46669,13.48840


In [5]:
# save the DataFrame as CSV file
berlin_df.to_csv("berlin_neighborhoods.csv", index=False)

#### Visualise map with neighborhoods superimposed

In [6]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [7]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

#### Using Foursquare API

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
radius = 2000
LIMIT = 100

venues = []
for lat, long, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [10]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5424, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlershof,52.43779,13.54778,Mia Toscana,52.438327,13.549573,Italian Restaurant
1,Adlershof,52.43779,13.54778,Griechisches Restaurant Athen,52.434901,13.546680,Greek Restaurant
2,Adlershof,52.43779,13.54778,Olympia Greek Food,52.433982,13.538500,Greek Restaurant
3,Adlershof,52.43779,13.54778,Food - Taste of the world on the road,52.428982,13.537843,Food Truck
4,Adlershof,52.43779,13.54778,Trattoria Il Mondo,52.434788,13.543176,Trattoria/Osteria


In [11]:
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 339 uniques categories.


In [12]:
#### Analyse each neighborhood
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped.head()

(5424, 340)
(96, 340)


,Neighborhoods,ATM,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridge,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Canal Lock,Candy Store,Capitol Building,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Street 

In [13]:
len(grouped[grouped["Italian Restaurant"] > 0])
berlin_rest = grouped[["Neighborhoods","Italian Restaurant"]]
berlin_rest

,Neighborhoods,Italian Restaurant
0,Adlershof,0.021277
1,Alt-Hohenschönhausen,0.000000
2,Alt-Treptow,0.020000
3,Altglienicke,0.000000
4,Baumschulenweg,0.000000
5,Biesdorf (Berlin),0.034483
6,Blankenburg (Berlin),0.000000
7,Blankenfelde,0.000000
8,Bohnsdorf,0.000000
9,Borsigwalde,0.036585


#### Clustering

In [129]:
kclusters = 4

berlin_clustering = berlin_rest.drop(["Neighborhoods"], 1)
berlin_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(berlin_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 0, 0, 2, 0, 0, 0, 2], dtype=int32)

In [130]:
merged = berlin_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,Italian Restaurant,Category
0,Adlershof,0.021277,2
1,Alt-Hohenschönhausen,0.000000,0
2,Alt-Treptow,0.020000,2
3,Altglienicke,0.000000,0
4,Baumschulenweg,0.000000,0


In [131]:
dfmerged = merged.merge(berlin_df)
dfmerged.head()

#Sort
dfmerged.sort_values(["Category"], inplace=True, ascending=False)
dfmerged

,Neighborhood,Italian Restaurant,Category,Latitude,Longitude
48,Lichtenrade,0.062500,3,52.39159,13.41247
21,Friedrichsfelde,0.042857,3,52.50527,13.51326
37,Hermsdorf (Berlin),0.058824,3,52.61488,13.31752
39,Karlshorst,0.042553,3,52.48181,13.52547
42,Kladow,0.071429,3,52.45301,13.14256
46,Lankwitz,0.049180,3,52.43895,13.35340
49,Lichterfelde (Berlin),0.057471,3,52.43293,13.31894
51,Mahlsdorf,0.068966,3,52.51297,13.61185
53,Mariendorf,0.054795,3,52.43855,13.38779
54,Marienfelde,0.064516,3,52.41262,13.36732


#### Visualising Clusters

In [133]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfmerged['Latitude'], dfmerged['Longitude'], dfmerged['Neighborhood'], dfmerged['Category']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

### Category 1: Neighborhoods with very low number of restaurants

In [134]:
merged.loc[merged['Category'] == 0]

,Neighborhood,Italian Restaurant,Category
1,Alt-Hohenschönhausen,0.000000,0
3,Altglienicke,0.000000,0
4,Baumschulenweg,0.000000,0
6,Blankenburg (Berlin),0.000000,0
7,Blankenfelde,0.000000,0
8,Bohnsdorf,0.000000,0
11,Buch (Berlin),0.000000,0
16,Falkenberg (Berlin),0.000000,0
17,Falkenhagener Feld,0.000000,0
19,Französisch Buchholz,0.000000,0


### Category 2: Neighborhoods with low number of restaurants

In [135]:
merged.loc[merged['Category'] == 1]

,Neighborhood,Italian Restaurant,Category
13,Charlottenburg,0.110000,1
28,Grunewald,0.130000,1
31,Halensee,0.120000,1
44,Kreuzberg,0.090000,1
72,Rudow,0.090909,1
74,Schmargendorf,0.145161,1
75,Schmöckwitz,0.100000,1
85,Waidmannslust,0.085714,1


### Category 3: Neighborhoods with a significant number of restaurants

In [136]:
merged.loc[merged['Category'] == 2]

,Neighborhood,Italian Restaurant,Category
0,Adlershof,0.021277,2
2,Alt-Treptow,0.020000,2
5,Biesdorf (Berlin),0.034483,2
9,Borsigwalde,0.036585,2
10,Britz,0.025641,2
18,Fennpfuhl,0.030000,2
23,Friedrichshain,0.030000,2
24,Frohnau,0.033333,2
30,Hakenfelde,0.030303,2
32,Hansaviertel,0.030000,2


### Category 4: Neighborhoods crowded with restaurants

In [137]:
merged.loc[merged['Category'] == 3]

,Neighborhood,Italian Restaurant,Category
12,Buckow (Berlin),0.065217,3
14,Charlottenburg-Nord,0.045977,3
15,Dahlem (Berlin),0.075949,3
20,Friedenau,0.070000,3
21,Friedrichsfelde,0.042857,3
27,Gropiusstadt,0.063492,3
37,Hermsdorf (Berlin),0.058824,3
39,Karlshorst,0.042553,3
42,Kladow,0.071429,3
46,Lankwitz,0.049180,3


### Observations

It is clear the Category 3 are very crowded with Italian Restaurants, and hence, Category 2 would be the best bet for opening a new restuarant because of not too much competition in these regions, but still a proven market. Client with USPs to stand out from the competition can also open new restaurants in neighborhoods in Cluster 1 with moderate competition. 